# Hyper Parameter Tuning

### To decide the Number of Hidden Layers and the No of neurons in those layers

In [2]:
!pip install tensorflow

  Using cached tensorflow-2.12.0-cp310-cp310-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp310-cp310-win_amd64.whl (272.8 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ---------------------------------------- 2.6/2.6 MB 6.7 MB/s eta 0:00:00
  Using cached jax-0.4.12.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached libclang-16.0.0-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.wh

In [5]:
pip install keras_tuner

     -------------------------------------- 176.1/176.1 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import RandomSearch

In [35]:
df = pd.read_csv('Real_Combine.csv')

In [36]:
df.isnull().sum()
# 1 null in PM 2.5.

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

In [37]:
df = df.fillna(df['PM 2.5'].mean())

In [38]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [39]:
# separating x and y features.
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

# Building custom function for the tuning

#### Hyperparameters below:
1. How many hidden layers?
2. How many neurons in each layer?
3. Learning rate?

In [52]:
def build_model(hp):
    model = keras.Sequential()
    # creating hidden layers (2 to 20 layers)
    for i in range(hp.Int('num_layers',2,20)):
        # no of neurons in each hidden layer.
        model.add(layers.Dense(units=hp.Int('units_'+str(i),
                                           min_value=32,
                                           max_value=512,
                                           step=32),
                              activation='relu'))
    # adding the output layer: linear for regression problems.
    model.add(layers.Dense(1,activation='linear'))
    # finally optimizing the model using Adam optimizer
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice(
    'learning_rate',[1e-2,1e-3,1e-4])),
                 loss='mean_absolute_error',
                 metrics=['mean_absolute_error'])
    return model
        

In [53]:
tuner = RandomSearch(
build_model,
objective='val_mean_absolute_error',
max_trials=12,
executions_per_trial=3,
directory='project1',
project_name='Air Quaility Index')

INFO:tensorflow:Reloading Tuner from project1\Air Quaility Index\tuner0.json


In [42]:
tuner.search_space_summary()

Search space summary
Default search space size: 22
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

# train test split

In [54]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [55]:
# tuner is random search object
tuner.search(x_train,y_train,epochs=15,
            validation_data=(x_test,y_test))

Trial 12 Complete [00h 00m 27s]
val_mean_absolute_error: 61.29922231038412

Best val_mean_absolute_error So Far: 44.214717864990234
Total elapsed time: 00h 04m 47s
INFO:tensorflow:Oracle triggered exit


In [56]:
# shows 10 best trials and their paramters.
tuner.results_summary()

Results summary
Results in project1\Air Quaility Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 10 summary
Hyperparameters:
num_layers: 6
units_0: 320
units_1: 352
learning_rate: 0.01
units_2: 64
units_3: 32
units_4: 32
units_5: 256
units_6: 320
units_7: 256
units_8: 512
units_9: 320
units_10: 64
units_11: 320
units_12: 32
units_13: 32
units_14: 384
units_15: 256
units_16: 448
units_17: 160
units_18: 416
units_19: 480
Score: 44.214717864990234

Trial 07 summary
Hyperparameters:
num_layers: 4
units_0: 96
units_1: 32
learning_rate: 0.01
units_2: 256
units_3: 256
units_4: 416
units_5: 416
units_6: 480
units_7: 160
units_8: 128
units_9: 416
units_10: 32
units_11: 64
units_12: 480
units_13: 416
units_14: 256
units_15: 192
units_16: 32
units_17: 224
units_18: 64
units_19: 96
Score: 46.13557052612305

Trial 09 summary
Hyperparameters:
num_layers: 13
units_0: 256
units_1: 512
learning_rate: 0.01
units_2: 480
units_3: 192
units_4: 384
units_5: 160

In [57]:
# we can take those paramters of the best trials. and train the model again on them.